In [5]:
import yaml
from pathlib import Path  
import shutil

In [2]:
base_dirs = ['']

In [3]:
def parse_config(yaml_path):
    """Parse YAML config file, accepting only basic YAML tags"""
    with open(yaml_path, "r") as stream:
        data = yaml.safe_load(stream)
    return data

def write_config(data,yaml_path):
    """Write YAML config file, using only basic YAML tags"""
    with open(yaml_path, mode="wt",encoding="utf-8") as stream:
        yaml.safe_dump(data,stream,sort_keys=False)

In [4]:
def find_hsi_datasets(base_dir:Path,dataset_subdir_search_str='0_raw'):
    subdirs = base_dir.rglob(f'**{dataset_subdir_search_str}')
    return [subdir.parent for subdir in subdirs]

In [ ]:
# "Dry-run", show found dataset dirs
for base_dir in base_dirs:
    dataset_dirs = find_hsi_datasets(Path(base_dir)):
    print(dataset_dirs)

In [ ]:
# Copy template to all dataset dirs
for base_dir in base_dirs:
    dataset_dirs = find_hsi_datasets(Path(base_dir)):
    for dataset_dir in dataset_dirs:
        shutil.copy('config.seabee.yaml',dataset_dir)